# Obtaining the pretrained model and Converting to TF-Lite


In [ ]:
import tensorflow as tf
import os
import pathlib
import numpy as np
import cv2
import glob

In [ ]:
# Cloning the tensorflow models 
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# Downloading & extracting the inbuilt SSD-Mobilenet V2 model
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
!tar -xvf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

In [ ]:
# Generates an intermediate SavedModel that can be used with the TFLite Converter
!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint \
    --pipeline_config_path ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config \
    --output_directory `pwd`



---

---







**CONVERSION**

In [ ]:
dir = '/content/saved_model'

1) Without Any Quantisation

In [ ]:
# Convert the model to TF lite
converter = tf.lite.TFLiteConverter.from_saved_model(dir)
tflite_model = converter.convert()

# Serialize the model
open('tflite_model_nq.tflite', 'wb').write(tflite_model)



---





2) Dynamic Range Quantisation

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_dr = converter.convert()
open('tflite_model_dr.tflite', 'wb').write(tflite_model_dr)



---






3) Full Integer Quantisation

In [ ]:
def rep_data_gen():
  a = []
  for img in glob.glob("/content/sample/*.jpg"):
    data = cv2.imread(img)
    img = cv2.resize(data, (300, 300))
    img = img / 255.0
    img = img.astype(np.float32)
    a.append(img)
    a1 = np.array(a)
    img = tf.data.Dataset.from_tensor_slices(a1).batch(1)
    for i in img:
        yield [i]

In [ ]:
# In onder to do full integer quantization, a range should be estimated of all the floating point tensors in the model
# For variable tensors, the convertor needs a representative dataset with a few inference cycles to calibrate
# Loop over every image and perform detection
converter_int = tf.lite.TFLiteConverter.from_saved_model(dir)
converter_int.representative_dataset = rep_data_gen
converter_int.optimizations = [tf.lite.Optimize.DEFAULT] # Using the default optimisation strategy
converter_int.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] # Ensures that TFlite interpreter only uses integers
converter_int.inference_input_type = tf.uint8  # Setting input type to be int8
converter_int.inference_output_type = tf.uint8  # Setting output type to be int8
tflite_quant_model_int = converter_int.convert() # Converts a saved model into TensorFlow Lite model
open('tflite_quant_model_int.tflite', 'wb').write(tflite_quant_model_int)



---



---



4) Float-Fallback Quantisation

In [ ]:
# Path to the directory of the saved model
converter_ff = tf.lite.TFLiteConverter.from_saved_model(dir)
# Assigning the representative dataset generator 
converter_ff.representative_dataset = rep_data_gen 
# Using the default optimisation strategy
converter_ff.optimizations = [tf.lite.Optimize.DEFAULT] 

# Converts a saved model into TensorFlow Lite model
tflite_quant_model_ff = converter_ff.convert() 
open('tflite_quant_model_ff.tflite', 'wb').write(tflite_quant_model_ff)



---



---



5) Float16 Quantisation

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model_f16 = converter.convert()
open('tflite_quant_model_f16.tflite', 'wb').write(tflite_quant_model_f16)